In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

In [2]:
# loading the chestmnist dataset
data_path = "data"
data = np.load(f"{data_path}/chestmnist_64.npz")

train_images = data['train_images']
train_labels = data['train_labels']

validation_images = data['val_images']
validation_labels = data['val_labels']

test_images = data['test_images']
test_labels = data['test_labels']


In [3]:
print(data)

NpzFile 'data/chestmnist_64.npz' with keys: train_images, train_labels, val_images, val_labels, test_images...


In [4]:
print(train_images[0].shape)

(64, 64)


In [5]:
print(type(train_images))

<class 'numpy.ndarray'>


In [6]:
print(f"The shape of train images: {train_images.shape}, train labels: {train_labels.shape}")
print(f"The shape of validation images: {validation_images.shape}, validation labels: {validation_labels.shape}")
print(f"The shape of test images: {test_images.shape}, test labels: {test_labels.shape}")


The shape of train images: (78468, 64, 64), train labels: (78468, 14)
The shape of validation images: (11219, 64, 64), validation labels: (11219, 14)
The shape of test images: (22433, 64, 64), test labels: (22433, 14)


In [7]:
class ChestMNISTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self): 
        return len(self.images) # number of samples
    
    def __getitem__(self, idx):
        img = self.images[idx]

        if self.transform: # apply transformations
            img = self.transform(img)

        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.float)
        
        return img, label

In [8]:
from torchvision import transforms # to apply transformations to the images

transforms = transforms.Compose([
    transforms.ToTensor(), # convert to tensor
    transforms.Normalize((0.5,), (0.5,)) # normalize the images
])

In [9]:
train_dataset = ChestMNISTDataset(train_images, train_labels, transform=transforms)
validation_dataset = ChestMNISTDataset(validation_images, validation_labels, transform=transforms)
test_dataset = ChestMNISTDataset(test_images, test_labels, transform=transforms)

In [10]:
# creating the dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=1000, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [12]:
model = models.resnet18(weights=None) # not pretrained
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False) # Change input channels from 3 to 1 (grayscale images)
model.fc = nn.Linear(512, 14)  # final fully connected layer

model = model.to(device) # move the model to the GPU

In [13]:
loss = nn.BCEWithLogitsLoss()  # for multi-label classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [23]:
training_epochs = 10

In [ ]:
for epoch in range(training_epochs):
    model.train() # set the model to training mode
    total_loss = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):

        inputs, targets = inputs.to(device), targets.to(device) # move to the GPU
        optimizer.zero_grad() # zero the gradients
        outputs = model(inputs) # forward pass
        loss_value = loss(outputs, targets) # compute loss
        loss_value.backward() # backward pass
        optimizer.step() # update weights

        total_loss += loss_value.item()


    avg_loss = total_loss / len(train_loader) # average loss for the epoch
    print(f"Epoch [{epoch+1}/{training_epochs}], Avg Loss: {avg_loss:.4f}")

Epoch [1/10], Avg Loss: 0.0997
Epoch [2/10], Avg Loss: 0.0803
Epoch [3/10], Avg Loss: 0.0615
Epoch [4/10], Avg Loss: 0.0459
Epoch [5/10], Avg Loss: 0.0358


In [ ]:
model.eval() # set the model to evaluation mode
with torch.no_grad():
    all_predictions = []
    all_labels = []
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions = torch.sigmoid(outputs) > 0.5
        
        all_predictions.append(predictions.cpu())
        all_labels.append(targets.cpu())
    
    all_predictions = torch.cat(all_predictions)
    all_labels = torch.cat(all_labels)
    
    accuracy = (all_predictions == all_labels).float().mean().item()
    print(f"Train Accuracy: {accuracy:.4f}")

Train Accuracy: 0.9651


In [ ]:
model.eval() # set the model to evaluation mode
with torch.no_grad():
    all_predictions = []
    all_labels = []
    
    for inputs, targets in validation_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions = torch.sigmoid(outputs) > 0.5
        
        all_predictions.append(predictions.cpu())
        all_labels.append(targets.cpu())
    
    all_predictions = torch.cat(all_predictions)
    all_labels = torch.cat(all_labels)
    
    accuracy = (all_predictions == all_labels).float().mean().item()
    print(f"validation Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9430


In [ ]:
model.eval() # set the model to evaluation mode
with torch.no_grad():
    all_predictions = []
    all_labels = []
    
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions = torch.sigmoid(outputs) > 0.5
        
        all_predictions.append(predictions.cpu())
        all_labels.append(targets.cpu())
    
    all_predictions = torch.cat(all_predictions)
    all_labels = torch.cat(all_labels)
    
    accuracy = (all_predictions == all_labels).float().mean().item()
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9413
